# Build an Agent

By themselves, language models can't take actions - they just output text.
A big use case for LangChain is creating **agents**.
[Agents](/docs/concepts/agents) are systems that use [LLMs](/docs/concepts/chat_models) as reasoning engines to determine which actions to take and the inputs necessary to perform the action.
After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via [tool-calling](/docs/concepts/tool_calling).

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## End-to-end agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!

In [ ]:
# Load environment variables from .env file
import dotenv
import os
dotenv.load_dotenv()

# Import LangChain components for building the agent
from langchain_openai import ChatOpenAI  # OpenAI's language model interface
from langchain_community.tools.tavily_search import TavilySearchResults  # Web search tool
from langchain_core.messages import HumanMessage  # Message structure for human input
from langgraph.checkpoint.memory import MemorySaver  # Memory persistence for conversations
from langgraph.prebuilt import create_react_agent  # Pre-built ReAct agent factory

# Initialize memory for conversation state persistence
memory = MemorySaver()

# Configure the language model (using OpenAI's GPT-4o-mini)
# Alternative Anthropic Claude model is commented out
# model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
model = ChatOpenAI(model_name='gpt-4o-mini')

# Configure the search tool with maximum 2 results per query
search = TavilySearchResults(max_results=2)

# Create tools list (can add more tools later)
tools = [search]

# Create the ReAct agent with model, tools, and memory checkpointing
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# Test the agent with a simple greeting message
# Configure thread ID for conversation memory persistence
config = {"configurable": {"thread_id": "abc123"}}

# Stream the agent's response in real-time
# The agent processes the human message and responds appropriately
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im chadi! and i live in byblos, lebanon")]},
    config,
    stream_mode="values",  # Stream complete message values
):
    # Pretty print the last message in each step
    step["messages"][-1].pretty_print()

In [ ]:
# Demonstrate agent tool usage with a context-aware query
# The agent remembers that Chadi lives in Byblos, Lebanon from the previous message
# However, the search seems to default to San Francisco (possibly a model confusion)
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,  # Same thread ID to maintain conversation context
    stream_mode="values",
):
    # Display the conversation flow:
    # 1. Human asks about weather
    # 2. AI decides to use search tool
    # 3. Tool returns search results
    # 4. AI synthesizes response from search results
    step["messages"][-1].pretty_print()

## Setup

### Jupyter Notebook

This guide (and most of the other guides in the documentation) uses [Jupyter notebooks](https://jupyter.org/) and assumes the reader is as well. Jupyter notebooks are perfect interactive environments for learning how to work with LLM systems because oftentimes things can go wrong (unexpected output, API down, etc), and observing these cases is a great way to better understand building with LLMs.

This and other tutorials are perhaps most conveniently run in a Jupyter notebook. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation

To install LangChain run:

In [ ]:
# Install required LangChain packages using pip
# This installs the core LangChain ecosystem components needed for building agents:
# - langchain-community: Community-contributed tools and integrations
# - langgraph: Graph-based agent framework for complex workflows
# - langchain-anthropic: Integration with Anthropic's Claude models
# - tavily-python: Web search API integration
# - langgraph-checkpoint-sqlite: SQLite-based memory persistence
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

For more details, see our [Installation guide](/docs/how_to/installation).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

### Tavily

We will be using [Tavily](/docs/integrations/tools/tavily_search) (a search engine) as a tool.
In order to use it, you will need to get and set an API key:

```bash
export TAVILY_API_KEY="..."
```

Or, if in a notebook, you can set it with:

```python
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()
```

## Define tools

We first need to create the tools we want to use. Our main tool of choice will be [Tavily](/docs/integrations/tools/tavily_search) - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.


In [ ]:
# Import and configure the Tavily search tool
from langchain_community.tools.tavily_search import TavilySearchResults

# Create search tool instance with max 2 results per query
search = TavilySearchResults(max_results=2)

# Test the search tool directly to see its output format
search_results = search.invoke("what is the weather in SF")
print(search_results)

# Additional tools can be created and added here as needed
# Once we have all the tools we want, we put them in a list for the agent
tools = [search]

## Using Language Models

Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4"}}} />


In [ ]:
# Configure alternative language model (Anthropic Claude)
# This cell shows how to use Claude instead of OpenAI
# Note: This is commented out in actual execution
from langchain_anthropic import ChatAnthropic

# Initialize Claude-3 Sonnet model
model = ChatAnthropic(model="claude-3-sonnet-20240229")

You can call the language model by passing in a list of messages. By default, the response is a `content` string.

In [ ]:
# Test basic language model interaction without tools
from langchain_core.messages import HumanMessage

# Send a simple message to the model and get response
response = model.invoke([HumanMessage(content="hi!")])

# Extract and display the text content of the response
response.content

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [ ]:
# Bind tools to the language model to enable tool-calling capabilities
# This gives the model awareness of available tools and their schemas
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [ ]:
# Test the tool-enabled model with a non-tool requiring message
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

# Examine both content and tool_calls fields
# For simple greetings, expect normal text response with no tool calls
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

Now, let's try calling it with some input that would expect a tool to be called.

In [ ]:
# Test the tool-enabled model with a query that requires tool usage
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

# Notice how the model now:
# 1. Has empty content (no direct text response)
# 2. Has populated tool_calls with search query
# This shows the model recognized the need to use the search tool
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create the agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using [LangGraph](/docs/concepts/architecture/#langgraph) to construct the agent. 
Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.


Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [ ]:
# Create a ReAct (Reasoning and Acting) agent using LangGraph
from langgraph.prebuilt import create_react_agent

# Initialize the agent executor with:
# - model: The language model (without pre-bound tools)
# - tools: List of available tools (search in this case)
# Note: create_react_agent will automatically bind tools to the model
agent_executor = create_react_agent(model, tools)

## Run the agent

We can now run the agent with a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions). Note that the agent will return the **final** state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [ ]:
# Test agent with a simple greeting (no tools needed)
# The invoke method runs the agent synchronously and returns final state
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

# The response contains all messages in the conversation:
# 1. Human message (input)
# 2. AI message (response)
# Note: No tool calls occurred since greeting doesn't require search
response["messages"]

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

Let's now try it out on an example where it should be invoking the tool

In [ ]:
# Test agent with a query requiring tool usage (weather information)
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)

# The response now contains 4 messages showing the complete ReAct cycle:
# 1. HumanMessage: User's weather query
# 2. AIMessage: Agent decides to use search tool (tool_calls populated)
# 3. ToolMessage: Search results from Tavily API
# 4. AIMessage: Agent synthesizes final answer from search results
response["messages"]

We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.

## Streaming Messages

We've seen how the agent can be called with `.invoke` to get  a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [ ]:
# Demonstrate streaming agent execution to show intermediate steps
# stream_mode="values" returns complete message states as they're updated
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    # Print each message as it becomes available:
    # 1. First shows human message
    # 2. Then shows AI deciding to use tool
    # 3. Then shows tool execution results
    # 4. Finally shows AI's synthesized response
    step["messages"][-1].pretty_print()

## Streaming tokens

In addition to streaming back messages, it is also useful to stream back tokens.
We can do this by specifying `stream_mode="messages"`.


::: note

Below we use `message.text()`, which requires `langchain-core>=0.3.37`.

:::

In [ ]:
# Demonstrate token-level streaming for real-time response generation
# stream_mode="messages" provides individual message chunks as they're generated
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    # Filter to only show text tokens from the agent's final response
    # langgraph_node="agent" ensures we only see the LLM's output, not tool calls
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        # Print each token as it's generated, separated by | for visibility
        print(text, end="|")

## Adding in memory

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [ ]:
# Initialize memory system for conversation persistence
from langgraph.checkpoint.memory import MemorySaver

# MemorySaver stores conversation state in memory (lost when session ends)
# For production, consider using persistent storage like SQLite or Redis
memory = MemorySaver()

In [ ]:
# Recreate agent with memory capabilities
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Configure conversation thread ID for memory persistence
# Same thread_id = same conversation; different thread_id = new conversation
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
# Test memory functionality by introducing the agent to a user
# This information should be remembered for subsequent interactions
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    # Display each step of the agent's response
    # Agent acknowledges the introduction and stores the name "Bob"
    print(chunk)
    print("----")

In [ ]:
# Test memory recall by asking the agent about previously shared information
# Using the same config (thread_id) ensures conversation continuity
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    # Agent successfully recalls the name "Bob" from previous interaction
    # This demonstrates working conversational memory
    print(chunk)
    print("----")

Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If you want to start a new conversation, all you have to do is change the `thread_id` used

In [ ]:
# Demonstrate conversation isolation with a different thread_id
# New thread = fresh conversation with no memory of previous interactions
config = {"configurable": {"thread_id": "xyz123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    # Agent doesn't know the name since this is a different conversation thread
    # This proves memory isolation between different thread_ids
    print(chunk)
    print("----")

## Conclusion

That's a wrap! In this quick start we covered how to create a simple agent. 
We've then shown how to stream back a response - not only with the intermediate steps, but also tokens!
We've also added in memory so you can have a conversation with them.
Agents are a complex topic with lots to learn! 

For more information on Agents, please check out the [LangGraph](/docs/concepts/architecture/#langgraph) documentation. This has it's own set of concepts, tutorials, and how-to guides.

In [ ]:
# This empty cell can be used for additional experimentation
# Try adding more tools, testing different queries, or exploring agent configurations